#### [CH07_lane3](/home/hobs/code/hobs/nlpia-manuscript/manuscript/adoc/CH07_lane3.adoc)

#### 

In [ ]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_md')  # <1>
text = 'right ones in the right order you can nudge the world'
doc = nlp(text)
df = pd.DataFrame([
   {k: getattr(t, k) for k in 'text pos_'.split()}
   for t in doc])

#### 

In [ ]:
pd.get_dummies(df, columns=['pos_'], prefix='', prefix_sep='')

#### .A function to compute Pierson correlation

In [ ]:
def corr(a, b):
   """ Compute the Pearson correlation coefficient R """
   a = a - np.mean(a)
   b = b - np.mean(b)
   return sum(a * b) / np.sqrt(sum(a*a) * sum(b*b))
a = np.array([0, 1, 2, 0, 1, 2, 0, 1, 2])
b = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
corr(a, b)

#### .A function to compute Pierson correlation

In [ ]:
corr(a, a)

#### .Tagging a quote with parts of speech

In [ ]:
nlp = spacy.load('en_core_web_md')
quote = "The right word may be effective, but no word was ever" \
   " as effective as a rightly timed pause."
tagged_words = {
   t.text: [t.pos_, int(t.pos_ == 'ADV')]  # <1>
   for t in nlp(quote)}
df_quote = pd.DataFrame(tagged_words, index=['POS', 'ADV'])
print(df_quote)

#### .Tagging a quote with parts of speech

In [ ]:
inpt = list(df_quote.loc['ADV'])
print(inpt)

#### .Tagging a quote with parts of speech

In [ ]:
kernel = [.5, .5]  # <1>
output = []
for i in range(len(inpt) - 1):  # <2>
   z = 0
   for k, weight in enumerate(kernel):  # <3>
       z = z + weight * inpt[i + k]
   output.append(z)
print(f'inpt:\n{inpt}')
print(f'len(inpt): {len(inpt)}')
print(f'output:\n{[int(o) if int(o)==o else o for o in output]}')
print(f'len(output): {len(output)}')

#### .A line plot of input (is_adv) and output (adverbiness)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 120  # <1>
import seaborn as sns
sns.set_theme('paper')  # <2>
df = pd.DataFrame([inpt, output], index=['inpt', 'output']).T
ax = df.plot(style=['+-', 'o:'], linewidth=3)

#### .A line plot of input (is_adv) and output (adverbiness)

In [ ]:
def convolve(inpt, kernel):
   output = []
   for i in range(len(inpt) - len(kernel) + 1):  # <1>
       output.append(
           sum(
               [
                   inpt[i + k] * kernel[k]
                   for k in range(len(kernel))  # <2>
               ]
           )
       )
   return output

#### 

In [ ]:
tags = 'ADV ADJ VERB NOUN'.split()
tagged_words = [
   [tok.text] + [int(tok.pos_ == tag) for tag in tags]  # <1>
   for tok in nlp(quote)]  # <2>
df = pd.DataFrame(tagged_words, columns=['token'] + tags).T
print(df)

#### 

In [ ]:
import torch
x = torch.tensor(
    df.iloc[1:].astype(float).values,
    dtype=torch.float32)  # <1>
x = x.unsqueeze(0) # <2>

#### 

In [ ]:
kernel = pd.DataFrame(
          [[1, 0, 0.],
           [0, 0, 0.],
           [0, 1, 0.],
           [0, 0, 1.]], index=tags)
print(kernel)

#### .Running a single example through a convolutional layer

In [ ]:
y = np.array(conv.forward(x).detach()).squeeze()
df.loc['y'] = pd.Series(y)
df

#### .Download secret message

In [ ]:
from nlpia2.init import maybe_download
url = 'https://upload.wikimedia.org/wikipedia/' \

#### .Download secret message

In [ ]:
filepath = maybe_download(url)  # <1>
filepath

#### .Download secret message

In [ ]:
from scipy.io import wavfile
sample_rate, audio = wavfile.read(filepath)
print(f'sample_rate: {sample_rate}')
print(f'audio:\n{audio}')

#### .Download secret message

In [ ]:
pd.options.display.max_rows = 7
audio = audio[:sample_rate * 2]  # <1>
audio = np.abs(audio - audio.max() / 2) - .5  # <2>
audio = audio / audio.max()  # <3>
audio = audio[::sample_rate // 400]  # <4>
audio = pd.Series(audio, name='audio')
audio.index = 1000 * audio.index / sample_rate  # <5>
audio.index.name = 'time (ms)'
print(f'audio:\n{audio}')

#### .A dot detecting kernel

In [ ]:
kernel = [-1] * 24 + [1] * 24 + [-1] * 24  # <1>
kernel = pd.Series(kernel, index=2.5 * np.arange(len(kernel)))
kernel.index.name = 'Time (ms)'
ax = kernel.plot(linewidth=3, ylabel='Kernel weight')

#### .A dot detecting kernel

In [ ]:
kernel = np.array(kernel) / sum(np.abs(kernel))  # <1>
pad = [0] * (len(kernel) // 2)  # <2>
isdot = convolve(audio.values, kernel)
isdot =  np.array(pad[:-1] + list(isdot) + pad)  # <3>
df = pd.DataFrame()
df['audio'] = audio
df['isdot'] = isdot - isdot.min()
ax = df.plot()

#### .A dot detecting kernel

In [ ]:
isdot = np.convolve(audio.values, kernel, mode='same')  # <1>
df['isdot'] = isdot - isdot.min()
ax = df.plot()

#### .Loading news posts

In [ ]:
df = pd.read_csv(HOME_DATA_DIR / 'news.csv')
df = df[['text', 'target']]  # <1>
print(df)

#### .Learning token embeddings from scratch

In [ ]:
from torch import nn

embedding = nn.Embedding(
    num_embeddings=2000,  # <1>
    embedding_dim=64,  # <2>
    padding_idx=0)
----
<1> Your vocabulary must be the same as in your tokenizer.
<2> A total of 50–100 dimensions is fine for small vocabularies and corpora.

The embedding layer will be the first layer in your CNN.
That will convert your token IDs into their own unique 64D word vectors.
And backpropagation during training will adjust the weights in each dimension for each word to match 64 different ways words can be used to talk about newsworthy disasters.
These embeddings won’t represent the complete meanings of words the way the fastText and GloVe vectors did in chapter 6; in fact, they are good for only one thing—determining whether a post contains newsworthy disaster information or not.

Finally, you can train your CNN to see how well it will do on an extremely narrow dataset, like the Kaggle disaster posts dataset.
Those hours of work crafting a CNN will pay off with a super-fast training time and impressive accuracy.

[id=learn-your-embeddings, reftext={chapter}.{counter:listing}]
.Learning your embeddings from scratch
[source,python]
----
from nlpia2.ch07.cnn.train79 import Pipeline  # <1>

pipeline = Pipeline(
    vocab_size=2000,
    embeddings=(2000, 64),
    epochs=7,
    torch_random_state=433994,  # <2>
    split_random_state=1460940,
)

pipeline = pipeline.train()
----
<1> nlpia2/src/nlpia2/ch07/cnn/train79.py
<2> Sets random seeds, so others can reproduce your results

[source,text]
----
Epoch: 1, loss: 0.66147, Train accuracy: 0.61392, Test accuracy: 0.63648
Epoch: 2, loss: 0.64491, Train accuracy: 0.69712, Test accuracy: 0.70735
Epoch: 3, loss: 0.55865, Train accuracy: 0.73391, Test accuracy: 0.74278
Epoch: 4, loss: 0.38538, Train accuracy: 0.76558, Test accuracy: 0.77165
Epoch: 5, loss: 0.27227, Train accuracy: 0.79288, Test accuracy: 0.77690
Epoch: 6, loss: 0.29682, Train accuracy: 0.82119, Test accuracy: 0.78609
Epoch: 7, loss: 0.23429, Train accuracy: 0.82951, Test accuracy: 0.79003
----

After only seven passes through your training dataset, you achieved 79% accuracy on your test set.
On a modern laptop CPU, this should take less than a minute, and you kept the overfitting to a minimum by minimizing the total parameters in your model.
The CNN uses very few parameters compared to the embedding layer.

What happens if you continue the training for a bit longer?

[id=continue-training, reftext={chapter}.{counter:listing}]
.Continuing the training
[source,python]
----
pipeline.epochs = 13  # <1>
pipeline = pipeline.train()
----
<1> 7 + 13 will give you 20 total epochs of training.

[source,python]

#### 

In [ ]:
def describe_model(model):  # <1>
    state = model.state_dict()
    names = state.keys()
    weights = state.values()
    params = model.parameters()
    df = pd.DataFrame()
    df['name'] = list(state.keys())
    df['all'] = p.numel(),
    df['learned'] = [
        p.requires_grad  # <2>
        for p in params],  # <3>
    size=p.size(),
    )

#### .Making room for GloVe embeddings

In [ ]:
from torch import nn
embedding = nn.Embedding(
    num_embeddings=2000,  # <1>
    embedding_dim=50,  # <2>
    padding_idx=0)

#### .Making room for GloVe embeddings

In [ ]:
from nessvec.files import load_vecs_df
glove = load_vecs_df(HOME_DATA_DIR / 'glove.6B.50d.txt')
zeroes = [0.] * 50
embed = []
for tok in vocab:  # <1>
    if tok in glove.index:
        embed.append(glove.loc[tok])
    else:
        embed.append(zeros.copy())  # <2>
embed = np.array(embed)
embed.shape

#### .Making room for GloVe embeddings

In [ ]:
pd.Series(vocab)

#### .Initializing your embedding layer with GloVE vectors

In [ ]:
embed = torch.Tensor(embed)  # <1>
print(f'embed.size(): {embed.size()}')
embed = nn.Embedding.from_pretrained(embed, freeze=False)  # <2>
print(embed)
----
<1> Converts the pandas DataFrame to a torch.Tensor
<2> freeze=False allows your Embedding layer to fine tune your embeddings

==== Detecting meaningful patterns

How you say something—the order of the words—makes a big difference.
You combine words to create patterns that mean something significant to you, so you can convey that meaning to someone else.

If you want your machine to be a meaningful natural language processor, it will need to be able to detect more than just the presence or absence of particular tokens.
You want your machine to detect meaningful patterns hidden within word sequences.footnote:[_The International Association of Facilitators Handbook of Group Facilitation: Best Practices from the Leading Organization in Facilitation_,  Sandy Schuman (Ed.) (https://books.google.com/books?id=TgWsY7oSgtsC&lpg=PT35&dq=%22beneath%20the%20words%22%20empathy%20listening&pg=PT35#v=onepage&q=%22beneath%20the%20words%22%20empathy%20listening&f=false).]

Convolutions are the filters that bring out meaningful patterns from words.
And the best part is, you no longer have to hardcode these patterns into the convolutional kernel.
The training process will search for the best possible pattern-matching convolutions for your problem.
Each time you propagate the error from your labeled dataset back through the network (backpropagation), the optimizer will adjust the weights in each of your filters so that they get better and better at detecting meaning and classifying your text examples.

=== Robustifying your CNN with dropout

Most neural networks are susceptible to adversarial examples that trick them into outputting incorrect classifications or text.
And sometimes, neural networks are susceptible to changes as straightforward as synonym substitution, misspellings, or insertion of slang.
In certain cases, all it takes is a little “word salad”—nonsensical random words—to distract and confuse an NLP algorithm.
Humans know how to ignore noise and filter out distractors, but machines sometimes have trouble with this.

_Robust NLP_ is the study of approaches and techniques for building machines that are smart enough to handle unusual text from diverse sources.footnote:[See Robin Jia’s thesis on Robust NLP (https://robinjia.github.io/assets/pdf/robinjia_thesis.pdf) and his presentation with Kai-Wei Chang, He He, and Sameer Singh (https://robustnlp-tutorial.github.io).]
In fact, research on robust NLP may uncover paths toward artificial general intelligence.
Humans are able to learn new words and concepts from just a few examples, and we generalize well—not too much and not too little.
Machines need a little help.
And if you can figure out the “secret sauce” that makes us humans good at this, then you can encode it into your NLP pipelines.

One popular technique for increasing the robustness of neural networks is _random dropout_.
_Random dropout_, or just _dropout_, has become popular because of its ease and effectiveness.
Your neural networks will almost always benefit from a dropout layer.
A dropout layer randomly hides some of the neuron outputs from the neurons listening to them.
This causes that pathway in your artificial brain to go quiet and forces the other neurons to learn from the examples in front of it during that dropout.

It’s counterintuitive, but dropout helps your neural network to spread the learning around.
Without a dropout layer, your network will focus on the words, patterns, and convolutional filters that helped it achieve the greatest accuracy boost.
But you need your neurons to diversify their patterns so that your network can be robust against common variations on natural language text.

The best place in your neural network to install a dropout layer is close to the end, just before you run the fully connected linear layer that computes the predictions on a batch of data.
This vector of weights passing into your linear layer are the outputs from your CNN and pooling layers.
Each of these values represents a sequence of words, or patterns of meaning and syntax.
Hiding some of these patterns from your prediction layer forces your prediction layer to diversify its “thinking.”
Though your software isn’t really thinking about anything, it’s okay to anthropomorphize it a bit, if it helps you develop intuitions about why techniques like random dropout can improve your model’s accuracy.

== PyTorch CNN to process disaster toots

Now comes the fun part.
You are going to build a real-world CNN that can distinguish real-world news from sensationalism.
Your model can help you filter out culture war tweets, so you can focus on news from real war zones.

First, you will see where your new convolution layers fit into the pipeline.
Then, you’ll assemble all the pieces to train a CNN on a dataset of disaster tweets.
And if doom scrolling and disaster are not your thing, the CNN is easily adaptable to any labeled dataset of tweets.
You can even pick a hashtag that you like and use that as you target label.
Then, you can find tweets that match that hashtag topic even when the person who posted it doesn’t use hashtags.

=== Network architecture


#### .CNN hyperparameters

In [ ]:
class CNNTextClassifier(nn.Module):

    def __init__(self, embeddings):
        super().__init__()

        self.seq_len = 40  # <1>
        self.vocab_size = 10000  # <2>
        self.embedding_size = 50  # <3>
        self.out_channels = 5  # <4>
        self.kernel_lengths = [2, 3, 4, 5, 6]  # <5>
        self.stride = 1  # <6>
        self.dropout = nn.Dropout(0)  # <7>
        self.pool_stride = self.stride  # <8>
        self.conv_out_seq_len = calc_out_seq_len(  # <9>
            seq_len=self.seq_len,
            kernel_lengths=self.kernel_lengths,
            stride=self.stride,
            )
----
<1> N_: Assumes a maximum text length of 40 tokens
<2> V: The number of unique tokens (words) in your vocabulary
<3> E: The number of word embedding dimensions (kernel input channels)
<4> F: The number of filters (kernel output channels)
<5> K: The number of columns of weights in each kernel
<6> S: The number of time steps (tokens) to slide the kernel forward with each step
<7> D: The portion of convolution output to ignore, where 0 dropout increases overfitting
<8> P: Pooling strides greater than 1 will increase feature reduction
<9> C: Total convolutional output size based on kernel and pooling hyperparameters

As with your handcrafted convolutions earlier in this chapter, the sequence length is reduced by each convolutional operation, and the amount of shortening depends on the size of the kernel and the stride.
The PyTorch documentationfootnote:[https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html.] for a `Conv1d` layer provides this formula and a detailed explanation of the terms:

[source,python]
----
def calc_conv_out_seq_len(seq_len, kernel_len,
                          stride=1, dilation=1, padding=0):
    """
    L_out =     (L_in + 2 * padding - dilation * (kernel_size - 1) - 1)
            1 + _______________________________________________________
                                        stride
    """
    return (
        1 + (seq_len +
             2 * padding - dilation * (kernel_len - 1) - 1
            ) //
        stride
        )
----

As shown in listing <<cnn-embedding>>, our first CNN layer is an `nn.Embedding` layer that converts a sequence of word ID integers into a sequence of embedding vectors.
It has as many rows as you have unique tokens in your vocabulary, including the new padding token.
It also has a column for each dimension of the embedding vectors.
You can load these embedding vectors from GloVe or any other pretrained embeddings.

[id=cnn-embedding, reftext={chapter}.{counter:listing}]
.Initializing the CNN embedding
[source,python]
----
self.embed = nn.Embedding(
    self.vocab_size,  # <1>
    self.embedding_size,  # <2>
    padding_idx=0)
state = self.embed.state_dict()
state['weight'] = embeddings  # <3>

#### .Constructing convolution and pooling layers

In [ ]:
self.convolvers = []
self.poolers = []
total_out_len = 0
for i, kernel_len in enumerate(self.kernel_lengths):
    self.convolvers.append(
        nn.Conv1d(in_channels=self.embedding_size,
                  out_channels=self.out_channels,
                  kernel_size=kernel_len,
                  stride=self.stride))
    print(f'conv[{i}].weight.shape: {self.convolvers[-1].weight.shape}')
    conv_output_len = calc_conv_out_seq_len(
        seq_len=self.seq_len, kernel_len=kernel_len, stride=self.stride)
    print(f'conv_output_len: {conv_output_len}')
    self.poolers.append(
        nn.MaxPool1d(kernel_size=conv_output_len, stride=self.stride))
    total_out_len += calc_conv_out_seq_len(
        seq_len=conv_output_len, kernel_len=conv_output_len,
        stride=self.stride)
    print(f'total_out_len: {total_out_len}')
    print(f'poolers[{i}]: {self.poolers[-1]}')
print(f'total_out_len: {total_out_len}')
self.linear_layer = nn.Linear(self.out_channels * total_out_len, 1)
print(f'linear_layer: {self.linear_layer}')
----

Unlike in the previous examples, this time you’ll create multiple convolution and pooling layers.
You won’t layer them up, as is often done in computer vision; instead, you will concatenate the convolution and pooling outputs together.
This is effective because you’ve limited the dimensionality of your convolution and pooling output by performing global max pooling and keeping the number of output channels much smaller than the number of embedding dimensions.

You can use print statements to help debug mismatching matrix shapes for each layer of your CNN.
You want to make sure you don’t unintentionally create too many trainable parameters that cause more overfitting than you’d like.
Your pooling outputs each contain a sequence length of `1`, but they also contain `5` channels for the embedding dimensions combined during convolution, as shown in the following listing.
Therefore, the concatenated and pooled convolution output is a 5x5 tensor, which produces a 25D linear layer for the output tensor that encodes the meaning of each text.

[id=cnn-layer, reftext={chapter}.{counter:listing}]
.CNN layer shapes
[source,python]
----
conv[0].weight.shape: torch.Size([5, 50, 2])
conv_output_len: 39
total_pool_out_len: 1
poolers[0]: MaxPool1d(kernel_size=39, stride=1, padding=0, dilation=1,
    ceil_mode=False)
conv[1].weight.shape: torch.Size([5, 50, 3])
conv_output_len: 38
total_pool_out_len: 2
poolers[1]: MaxPool1d(kernel_size=38, stride=1, padding=0, dilation=1,
    ceil_mode=False)
conv[2].weight.shape: torch.Size([5, 50, 4])
conv_output_len: 37
total_pool_out_len: 3
poolers[2]: MaxPool1d(kernel_size=37, stride=1, padding=0, dilation=1,
    ceil_mode=False)
conv[3].weight.shape: torch.Size([5, 50, 5])
conv_output_len: 36
total_pool_out_len: 4
poolers[3]: MaxPool1d(kernel_size=36, stride=1, padding=0, dilation=1,
    ceil_mode=False)
conv[4].weight.shape: torch.Size([5, 50, 6])
conv_output_len: 35
total_pool_out_len: 5
poolers[4]: MaxPool1d(kernel_size=35, stride=1, padding=0, dilation=1,
     ceil_mode=False)
total_out_len: 5

#### 

In [ ]:
out_pool_total = 0  # <1>
for kernel_len, stride in zip(kernel_lengths, strides):
    out_conv = (
        (in_seq_len - dilation * (kernel_len - 1) - 1)
        // stride
        ) + 1
    out_pool_total +=  (
        (out_conv - dilation * (kernel_len - 1) - 1)
        // stride

#### .CNN hyperparameter tuning

In [ ]:
             learning  seq  case vocab           training      test
 kernel_sizes    rate  len  sens  size dropout  accuracy  accuracy
          [2]  0.0010   32 False  2000     NaN    0.5790    0.5459
[1 2 3 4 5 6]  0.0010   40 False  2000     NaN    0.7919    0.7100
    [2 3 4 5]  0.0015   40 False  2000     NaN    0.8038    0.7152
[1 2 3 4 5 6]  0.0010   40  True  2000     NaN    0.7685    0.7520
          [2]  0.0010   32  True  2000     0.2    0.8472    0.7533
    [2 3 4 5]  0.0010   32  True  2000     0.2    0.8727    0.7900
----

Can you find a better combination of hyperparameters to improve this model’s accuracy?
This is quite a difficult problem, so don’t expect to achieve much better than 80% test set accuracy.
Even human readers can’t reliably determine whether a tweet represents a factual newsworthy disaster or not.
After all, many humans (and bots) are composing these tweets in an attempt to fool readers.
This is an adversarial problem.
Even a small, one-layer CNN does a decent job.

[id=learning-curve, reftext={chapter}.{counter:figure}]
.The learning curve for the best hyperparameters we found
image::../images/ch07/learning-curve-87-79.png[alt="Training and test set accuracy for 10 epochs of training using the best hyperparameters found", width=80%, link="../images/ch07/learning-curve-87-79.png"]

The key to hyperparameter tuning is to conscientiously record each experiment and make thoughtful hyperparameter adjustments for the next experiment.
You can automate this decision-making with a Bayesian optimizer, but in most cases, using your own biological neural network—the intuition you develop with practice—for the Bayesian optimization will lead to faster hyperparameter tuning.
And if you are curious about the effect of the transpose operation on the embedding layer, you can try it both ways to see which works best on your problem, but you probably want to follow the experts if your goal is achieving state-of-the-art solutions to difficult problems.
Don’t believe everything you read on the internet, especially when it comes to CNNs for NLP.

== Test yourself

. For a length `3` kernel and an input array of length `8`, what is the length of the output?
. What is the kernel for detecting an SOS distress signalfootnote:[This initialism is short for _save our souls_ or _save our ship_.] within the secret message audio file in this chapter?
. What is the best training set accuracy you can achieve after tuning the hyperparameters for the newsworthiness microblog post problem?
. How would you extend the model to accommodate an additional class? The news.csv file, provided in the nlpia2 package on GitLab, contains famous quotes to give you another level of profundity to attempt to classify with your CNN.
. Write three kernels, one each for detecting dots, dashes, and pauses. Write a pooling function that counts unique occurrences of these symbols. BONUS: Create a system of functions that _translates_ the secret message audio file into the symbols `"."`, `"-"`, and `" "`.
. Find some hyperparameters (don’t forget about random seeds) that achieve better than 80% accuracy on the test set for the disaster tweets dataset.
. Create a sarcasm detector using word-based CNN with datasets and examples on Hugging Face (huggingface.co). Do you find the claim of detecting sarcasm with 91% accuracy, made in several published papers,footnote:[Ivan Helin makes the claim of 92% accuracy for their model on Hugging Face (https://huggingface.co/helinivan/english-sarcasm-detector).]footnote:[Soujanya Poria et al. claim an accuracy rate of 91% in “A Deeper Look into Sarcastic Tweets Using a CNN” (https://arxiv.org/abs/1610.08815).] from a single, context-free tweet to be credible? 

== Summary

* A convolution is a windowed filter that slides over your sequence of words to compress its meaning into an encoding vector.
* Handcrafted convolutional filters work great on predictable signals, such as Morse code, but you will need CNNs that learn their own filters for NLP.
* Neural networks can extract patterns in a sequence of words that other NLP approaches would miss.
* During training, if you sandbag your model a bit with a dropout layer, you can keep it from overachieving (over fitting) on your training data.
* Hyperparameter tuning for neural networks gives you more room to exercise your creativity than conventional machine learning models.